In [6]:
%pip install torch tqdm obonet biopython transformers scikit-learn -q

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Test path setup
import sys
from pathlib import Path

src_path = Path.cwd() / 'src'
print(f"Current directory: {Path.cwd()}")
print(f"Src path: {src_path}")
print(f"Src exists: {src_path.exists()}")

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
    print(f"Added to path: {src_path}")

print(f"\nPython path:")
for p in sys.path[:5]:
    print(f"  {p}")

Current directory: c:\Users\Olale\Documents\Codebase\Science\cafa-6-protein-function-prediction
Src path: c:\Users\Olale\Documents\Codebase\Science\cafa-6-protein-function-prediction\src
Src exists: True
Added to path: c:\Users\Olale\Documents\Codebase\Science\cafa-6-protein-function-prediction\src

Python path:
  c:\Users\Olale\Documents\Codebase\Science\cafa-6-protein-function-prediction\src
  c:\Users\Olale\Documents\Codebase\Science\src
  c:\Users\Olale\Documents\Codebase\Science\src
  c:\Users\Olale\Documents\Codebase\Science\src
  c:\Users\Olale\Documents\Codebase\Science\src


In [15]:
# Test importing our modules with corrected path
from data.loaders import OntologyLoader
print("✅ Successfully imported OntologyLoader")

ModuleNotFoundError: No module named 'data.loaders'

In [14]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Our modules
from data.loaders import OntologyLoader, LabelLoader, SequenceLoader
from data.finetune_dataset import FineTuneDataset
from models.esm_classifier import ESMForSequenceClassification
from inference.propagation import propagate_predictions, get_propagated_terms

print("✅ Imports successful")

ModuleNotFoundError: No module named 'data.loaders'

## 1. Load GO Ontology

The ontology is a directed graph where edges point from specific terms to general ones.

In [ ]:
base_dir = Path.cwd().parent

print("Loading GO ontology...")
ontology = OntologyLoader(base_dir / "Train" / "go-basic.obo")

print(f"Total GO terms: {len(ontology.graph)}")
print(f"\nGraph type: {type(ontology.graph)}")
print(f"Nodes: {ontology.graph.number_of_nodes()}")
print(f"Edges: {ontology.graph.number_of_edges()}")

## 2. Test Propagation with Example Terms

Let's check: if we predict "nuclease activity", what ancestors should we add?

In [ ]:
# Example: nuclease activity
nuclease = 'GO:0004518'

# Get all ancestors
ancestors = ontology.get_ancestors(nuclease)

print(f"Term: {nuclease}")
if nuclease in ontology.graph:
    print(f"Name: {ontology.graph.nodes[nuclease].get('name', 'N/A')}")
    print(f"Namespace: {ontology.get_namespace(nuclease)}")

print(f"\nNumber of ancestors: {len(ancestors)}")
print(f"\nFirst 10 ancestors:")
for anc in list(ancestors)[:10]:
    name = ontology.graph.nodes[anc].get('name', 'N/A')
    print(f"  {anc}: {name}")

## 3. Test Array Propagation

Simulate model predictions and see how propagation changes probabilities.

In [ ]:
# Create dummy predictions for 5 terms
test_terms = [
    'GO:0003674',  # molecular_function (root)
    'GO:0016787',  # hydrolase activity
    'GO:0004518',  # nuclease activity (child of hydrolase)
    'GO:0008150',  # biological_process (root)
    'GO:0006281',  # DNA repair
]

# Simulate: high confidence only for nuclease (0.9)
preds_before = np.array([[0.05, 0.10, 0.90, 0.03, 0.15]])

print("Before propagation:")
for term, prob in zip(test_terms, preds_before[0]):
    name = ontology.graph.nodes[term].get('name', 'N/A')
    print(f"  {term} ({name}): {prob:.3f}")

# Apply propagation
preds_after = propagate_predictions(preds_before, test_terms, ontology, strategy='max')

print("\nAfter propagation:")
for term, prob in zip(test_terms, preds_after[0]):
    name = ontology.graph.nodes[term].get('name', 'N/A')
    change = '✅' if prob > preds_before[0][test_terms.index(term)] else ''
    print(f"  {term} ({name}): {prob:.3f} {change}")

print(f"\n🎯 Hydrolase boosted: {preds_before[0][1]:.3f} → {preds_after[0][1]:.3f}")

## 4. Load Training Data

We need the validation set to measure F1 improvement.

In [ ]:
print("Loading sequences...")
seq_loader = SequenceLoader(base_dir / "Train" / "train_sequences.fasta")

print("Loading labels...")
label_loader = LabelLoader(base_dir / "Train" / "train_terms.tsv")

print("Loading IA weights...")
ia_df = pd.read_csv(base_dir / "IA.tsv", sep='\t')
ia_weights = dict(zip(ia_df['term'], ia_df['IA']))

print(f"\nTotal proteins: {len(seq_loader.sequences)}")
print(f"Total annotations: {len(label_loader.df)}")
print(f"IA weights available: {len(ia_weights)}")

## 5. Load Fine-Tuned Model

Our best model: ESM-2 8M with Asymmetric Loss (F1 = 0.2331)

In [ ]:
model_path = base_dir / "models" / "esm_finetuned" / "best_model"

print(f"Loading model from {model_path}...")

# Check if model exists
if not model_path.exists():
    print("❌ Model not found. Please train model first using src/training/finetune_esm.py")
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
    
    # Load model
    model = ESMForSequenceClassification.from_pretrained(str(model_path))
    model.to(device)
    model.eval()
    
    print(f"✅ Model loaded successfully")
    print(f"Output dimension: {model.classifier.out_features}")

## 6. Create Validation Dataset

Use same split as training (last 20% of data)

In [ ]:
print("Creating dataset...")
full_dataset = FineTuneDataset(
    sequence_loader=seq_loader,
    label_loader=label_loader,
    max_length=512,
    top_k_terms=5000
)

# Split (80/20)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

from torch.utils.data import Subset
val_dataset = Subset(full_dataset, range(train_size, len(full_dataset)))

print(f"Validation samples: {len(val_dataset)}")
print(f"Top terms vocabulary: {len(full_dataset.top_terms)}")

## 7. Evaluation Helper Functions

In [ ]:
def compute_f1_with_threshold(y_true, y_pred, threshold, ia_weights_dict, term_list):
    """
    Compute weighted F1 score.
    
    Args:
        y_true: (N, K) binary labels
        y_pred: (N, K) probabilities
        threshold: float, prediction threshold
        ia_weights_dict: dict mapping GO terms to IA weights
        term_list: list of GO term IDs
    """
    # Threshold predictions
    y_pred_binary = (y_pred >= threshold).astype(int)
    
    # Get IA weights for our vocabulary
    weights = np.array([ia_weights_dict.get(term, 1.0) for term in term_list])
    
    # Compute per-sample F1
    f1_scores = []
    precisions = []
    recalls = []
    
    for i in range(len(y_true)):
        true_pos = (y_true[i] == 1) & (y_pred_binary[i] == 1)
        pred_pos = (y_pred_binary[i] == 1)
        actual_pos = (y_true[i] == 1)
        
        # Weighted counts
        tp = (true_pos * weights).sum()
        fp = ((pred_pos & ~true_pos) * weights).sum()
        fn = ((actual_pos & ~true_pos) * weights).sum()
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        f1_scores.append(f1)
        precisions.append(precision)
        recalls.append(recall)
    
    return {
        'f1': np.mean(f1_scores),
        'precision': np.mean(precisions),
        'recall': np.mean(recalls)
    }

print("✅ Helper functions defined")

## 8. Generate Predictions on Validation Set

Get model predictions for all validation samples.

In [ ]:
from torch.utils.data import DataLoader

# Create dataloader
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

print("Generating predictions...")
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader, desc="Inference"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].cpu().numpy()
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # Convert to probabilities
        probs = torch.sigmoid(logits).cpu().numpy()
        
        all_preds.append(probs)
        all_labels.append(labels)

# Concatenate
all_preds = np.vstack(all_preds)
all_labels = np.vstack(all_labels)

print(f"\nPredictions shape: {all_preds.shape}")
print(f"Labels shape: {all_labels.shape}")

## 9. Evaluate WITHOUT Propagation

Baseline: our current best performance

In [ ]:
print("Evaluating WITHOUT propagation...")
print("Testing thresholds...\n")

thresholds = [0.01, 0.05, 0.10, 0.15, 0.20, 0.30, 0.40, 0.50]
results_baseline = []

for thr in thresholds:
    metrics = compute_f1_with_threshold(
        all_labels, 
        all_preds, 
        thr, 
        ia_weights, 
        full_dataset.top_terms
    )
    results_baseline.append({
        'threshold': thr,
        'f1': metrics['f1'],
        'precision': metrics['precision'],
        'recall': metrics['recall']
    })
    print(f"Thr={thr:.2f}: F1={metrics['f1']:.4f}, P={metrics['precision']:.4f}, R={metrics['recall']:.4f}")

# Find best
best_baseline = max(results_baseline, key=lambda x: x['f1'])
print(f"\n🏆 Best WITHOUT propagation:")
print(f"   F1 = {best_baseline['f1']:.4f} at threshold {best_baseline['threshold']}")

## 10. Apply Propagation

Add ancestor terms to predictions using GO hierarchy

In [ ]:
print("Applying label propagation...")

# Convert predictions to torch for propagation function
all_preds_torch = torch.from_numpy(all_preds)

# Propagate (this adds ancestor probabilities)
all_preds_propagated = propagate_predictions(
    all_preds_torch,
    full_dataset.top_terms,
    ontology,
    strategy='max'
)

# Convert back to numpy
all_preds_propagated = all_preds_propagated.numpy()

print(f"✅ Propagation complete")
print(f"Shape unchanged: {all_preds_propagated.shape}")

# Check how many probabilities increased
increased = (all_preds_propagated > all_preds).sum()
total = all_preds.size
print(f"Probabilities boosted: {increased:,} / {total:,} ({100*increased/total:.2f}%)")

## 11. Evaluate WITH Propagation

Key question: did F1 improve?

In [ ]:
print("Evaluating WITH propagation...")
print("Testing thresholds...\n")

results_propagated = []

for thr in thresholds:
    metrics = compute_f1_with_threshold(
        all_labels, 
        all_preds_propagated, 
        thr, 
        ia_weights, 
        full_dataset.top_terms
    )
    results_propagated.append({
        'threshold': thr,
        'f1': metrics['f1'],
        'precision': metrics['precision'],
        'recall': metrics['recall']
    })
    print(f"Thr={thr:.2f}: F1={metrics['f1']:.4f}, P={metrics['precision']:.4f}, R={metrics['recall']:.4f}")

# Find best
best_propagated = max(results_propagated, key=lambda x: x['f1'])
print(f"\n🏆 Best WITH propagation:")
print(f"   F1 = {best_propagated['f1']:.4f} at threshold {best_propagated['threshold']}")

## 12. Compare Results

Did we hit our target (F1 ≥ 0.25)?

In [ ]:
print("=" * 60)
print("📊 COMPARISON")
print("=" * 60)

print(f"\nBaseline (no propagation):")
print(f"  F1:        {best_baseline['f1']:.4f}")
print(f"  Precision: {best_baseline['precision']:.4f}")
print(f"  Recall:    {best_baseline['recall']:.4f}")
print(f"  Threshold: {best_baseline['threshold']}")

print(f"\nWith Propagation:")
print(f"  F1:        {best_propagated['f1']:.4f}")
print(f"  Precision: {best_propagated['precision']:.4f}")
print(f"  Recall:    {best_propagated['recall']:.4f}")
print(f"  Threshold: {best_propagated['threshold']}")

improvement = best_propagated['f1'] - best_baseline['f1']
pct_improvement = 100 * improvement / best_baseline['f1']

print(f"\n{'🎉' if improvement > 0 else '⚠️'} Improvement:")
print(f"  Δ F1:      {improvement:+.4f} ({pct_improvement:+.2f}%)")

if best_propagated['f1'] >= 0.25:
    print(f"\n✅ TARGET REACHED! F1 ≥ 0.25")
else:
    print(f"\n⚠️ Target not reached (goal: 0.25, got: {best_propagated['f1']:.4f})")

print("=" * 60)

## 13. Visualize Improvement

In [ ]:
# Plot F1 vs threshold
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: F1 curves
ax1 = axes[0]
baseline_f1s = [r['f1'] for r in results_baseline]
propagated_f1s = [r['f1'] for r in results_propagated]

ax1.plot(thresholds, baseline_f1s, 'o-', label='Without Propagation', linewidth=2)
ax1.plot(thresholds, propagated_f1s, 's-', label='With Propagation', linewidth=2)
ax1.axhline(y=0.25, color='red', linestyle='--', alpha=0.5, label='Target (0.25)')
ax1.set_xlabel('Threshold', fontsize=12)
ax1.set_ylabel('F1 Score', fontsize=12)
ax1.set_title('F1 Score vs Threshold', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Right: Precision-Recall trade-off
ax2 = axes[1]
baseline_prec = [r['precision'] for r in results_baseline]
baseline_rec = [r['recall'] for r in results_baseline]
propagated_prec = [r['precision'] for r in results_propagated]
propagated_rec = [r['recall'] for r in results_propagated]

ax2.plot(baseline_rec, baseline_prec, 'o-', label='Without Propagation', linewidth=2)
ax2.plot(propagated_rec, propagated_prec, 's-', label='With Propagation', linewidth=2)
ax2.set_xlabel('Recall', fontsize=12)
ax2.set_ylabel('Precision', fontsize=12)
ax2.set_title('Precision-Recall Curve', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Plots generated")

## 14. Detailed Example: How Propagation Works

Pick a random sample and show term-by-term changes

In [ ]:
# Pick a sample with decent number of predictions
sample_idx = 100

print(f"Sample #{sample_idx}")
print("=" * 80)

# Get predictions
preds_before = all_preds[sample_idx]
preds_after = all_preds_propagated[sample_idx]
true_labels = all_labels[sample_idx]

# Find terms where probability increased
increased_mask = preds_after > preds_before
increased_indices = np.where(increased_mask)[0]

print(f"\nTerms with boosted probability: {len(increased_indices)}\n")

# Show top 10 boosted terms
boosts = preds_after[increased_indices] - preds_before[increased_indices]
top_boost_idx = increased_indices[np.argsort(boosts)[-10:][::-1]]

for idx in top_boost_idx:
    term = full_dataset.top_terms[idx]
    name = ontology.graph.nodes[term].get('name', 'N/A') if term in ontology.graph else 'N/A'
    before = preds_before[idx]
    after = preds_after[idx]
    true_val = true_labels[idx]
    
    true_str = '✅' if true_val == 1 else ''
    print(f"{term} ({name[:40]}...)")
    print(f"  Before: {before:.4f} → After: {after:.4f} (+{after-before:.4f}) {true_str}")
    print()

## 15. Summary & Next Steps

In [ ]:
print("=" * 80)
print("📋 SUMMARY")
print("=" * 80)

print("\n✅ What we did:")
print("  1. Implemented label propagation using GO hierarchy")
print("  2. Evaluated on validation set with/without propagation")
print("  3. Measured F1 improvement")

print("\n📊 Results:")
print(f"  Baseline F1:     {best_baseline['f1']:.4f}")
print(f"  Propagated F1:   {best_propagated['f1']:.4f}")
print(f"  Improvement:     {improvement:+.4f} ({pct_improvement:+.2f}%)")

print("\n🎯 Next actions (from PLAN.md):")
print("  1. ✅ Label propagation (DONE)")
print("  2. ⬜ Per-aspect thresholds (MF/BP/CC separate)")
print("  3. ⬜ Simple ensemble (KNN + ESM)")
print("  4. ⬜ Larger backbone (ESM-2 35M)")

print("\n💡 Key insight:")
if improvement > 0:
    print("  Propagation helps! Ontology consistency = free performance.")
else:
    print("  Propagation may need tuning (try different strategies).")

print("\n" + "=" * 80)

## 16. Save Results

In [ ]:
# Save comparison to CSV
results_df = pd.DataFrame({
    'threshold': thresholds,
    'f1_baseline': [r['f1'] for r in results_baseline],
    'f1_propagated': [r['f1'] for r in results_propagated],
    'precision_baseline': [r['precision'] for r in results_baseline],
    'precision_propagated': [r['precision'] for r in results_propagated],
    'recall_baseline': [r['recall'] for r in results_baseline],
    'recall_propagated': [r['recall'] for r in results_propagated],
})

output_path = base_dir / "results" / "propagation_comparison.csv"
output_path.parent.mkdir(exist_ok=True)
results_df.to_csv(output_path, index=False)

print(f"✅ Results saved to {output_path}")
print("\n📊 Results preview:")
print(results_df)